## 요약 데이터 임베딩

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [2]:
news_df=pd.read_csv('./Data/news.csv')

news_df.dropna(axis=0,inplace=True)

nltk.download('stopwords')
nltk.download('puntk')
nltk.download('punkt_tab')


# 영어, 소문자로 통일, 불용어 제거, 숫자 제거(x), 특수문자 제거, 마지막에 내용이 줄임표시 되어있는 것들이 있음(...)

stop_words=set(stopwords.words('english'))  


def clean_text(text):
    text=text.lower()
    #text=re.sub(r'\d+','',text)
    text=re.sub(r'[^\w\s]','',text) # 특수문자 제거
    text=re.sub(r'\s+',' ',text) # 연속된 공백 하나의 공백으로
    tokens=word_tokenize(text)
    for token in tokens:
        if token in stop_words:
            tokens.remove(token)    
    text=' '.join(tokens)   
    
    return text
    
news_df['cleaned_abstract']=news_df['abstract'].apply(clean_text)
news_df[['cleaned_abstract','abstract']].head(10)
news_df.shape


[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading puntk: Package 'puntk' not found in index
[nltk_data] Downloading package punkt_tab to /Users/mac/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


(48612, 9)

In [6]:
pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tf-keras


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 MB 42.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.6/667.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.2
    Uninstalling tensorflow-2.16.2:
      Successfully uninstalled tensorflow-2.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Sentence Transformer 모델 로드 (임베딩 생성)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

#뉴스 요약 데이터 벡터화
embeddings = model.encode(news_df["cleaned_abstract"].tolist(), convert_to_numpy=True)

# FAISS Index 생성 (L2 거리 기반 검색)
dim = embeddings.shape[1]  # 벡터 차원 크기
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print(f"FAISS에 저장된 벡터 개수: {index.ntotal}")  


/Users/mac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mac/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS에 저장된 벡터 개수: 48612


In [9]:
# 저장

faiss.write_index(index, "./Data/news_faiss.index")


In [6]:
def search_news(query, top_k=5):
    """
    사용자의 검색어(query)와 가장 유사한 뉴스 요약을 찾음
    """
    query_embedding = model.encode([query]).astype(np.float32)  # 검색어 벡터화
    distances, indices = index.search(query_embedding, top_k)  # FAISS에서 검색

    print(f"검색어: {query}\n")
    for i, idx in enumerate(indices[0]):
        print(f" Top-{i+1} (유사도 점수: {distances[0][i]:.4f})")
        print(f"   {news_df.iloc[idx]['abstract']}\n")

# 테스트 실행
search_news("climate change impact")


검색어: climate change impact

 Top-1 (유사도 점수: 0.5713)
   The economic effects of global warming may arrive sooner and with a bigger impact than previously thought, according to Oxford Economics in a report that compares recent scientific research with the economic literature on the costs of climate change.

 Top-2 (유사도 점수: 0.6713)
   It aims to draw attention to climate change

 Top-3 (유사도 점수: 0.6948)
   Scientists from 153 countries have published a paper, using analysis of more than 40 years of data, warning of "untold human suffering" if action on climate change isn't taken, along with six immediate steps we must take to reduce planet-warming effects.

 Top-4 (유사도 점수: 0.7305)
   Humans are affecting Earth's climate, and there's a wealth of scientific evidence to support that fact. We've already begun to see some seriously troubling trends that may be associated with our altering of the climate, including mass die-offs of ocean coral, widespread drought, and increasingly powerful and u